In [8]:
import pandas as pd
import itertools

In [9]:
MIN_SUP = 0.02

In [10]:
data_file = "./apriori/75000/75000-out2.csv"

def parse_to_df(file_name):
    f = open(file_name)
    lines = f.readlines()
    first_line = lines[0].split(",")
    columns = [i-1 for i in range(1, len(first_line))]
    rows = []
    index = []
    for line in lines:
        splt = list(map(int, line.strip().split(",")))
        rows.append(splt[1:])
        index.append(splt[0])
    
    return pd.DataFrame(rows, columns=columns, index=index)

In [11]:
# given a set of frequent itemsets F and a candidate 
# frequent item set of size k, checks whether all
# k-1 size subsets are in F
def is_valid_candidate(F, u):
    for elem in u:
        if (u - {elem}) not in F:
            return False
    
    return True

# Given a set of frequent itemsets F and a size k,
# constructs all possible k+1 sized candidate itemsets
def candidate_gen(F, k):
    candidates = set()
    
    k_sized_sets = list(filter(lambda s: len(s) == k, F))
    for (first, second) in itertools.combinations(k_sized_sets, r=2):
        joined = first.union(second)
        if len(joined) == k+1 and is_valid_candidate(F, joined):
            candidates.add(frozenset(joined))

    return candidates

In [12]:
df = parse_to_df(data_file)

In [13]:
def check_subset(row, s):
    for elem in s:
        if row[elem] == 0:
            return False
    
    return True

In [14]:
def apriori(T, minSup):
    counts = {}
    flags = {}
    k = 2
    I = T.index
    n_rows = len(T.index)
    F_cur = {frozenset({i}) for i in T.columns if T[i].sum() / n_rows >= minSup}
    F = F_cur
    
    print(F_cur)
    #print(F)
    """
    print(n_rows)
    print([T[i].sum() / n_rows for i in T.columns])
    """
    while len(F_cur) > 0:
        for iset in F_cur:
            flags[iset] = True
            
        candidates = candidate_gen(F_cur, k-1)
        #print(candidates)
        for c in candidates:
            counts[c] = 0
        for idx in T.index:
            row = T.loc[idx]
            for c in candidates:
                if check_subset(row, c):
                    counts[c] += 1
       
        F_next = {c for c in candidates if counts[c] / n_rows >= minSup}
        for s1 in F_cur:
            for s2 in F_next:
                if s1.issubset(s2):
                    flags[s1] = False
                    
        F_cur = F_next
        F = F.union(F_cur)
        k += 1
    
    
    return {iset for iset in F if flags[iset]}

df = parse_to_df(data_file)
apriori(df, MIN_SUP)

{frozenset({40}), frozenset({32}), frozenset({49}), frozenset({43}), frozenset({28}), frozenset({10}), frozenset({25}), frozenset({1}), frozenset({18}), frozenset({30}), frozenset({36}), frozenset({33}), frozenset({23}), frozenset({39}), frozenset({14}), frozenset({27}), frozenset({24}), frozenset({8}), frozenset({45}), frozenset({13}), frozenset({31}), frozenset({16}), frozenset({41}), frozenset({22}), frozenset({48}), frozenset({11}), frozenset({17}), frozenset({44}), frozenset({9}), frozenset({26}), frozenset({47}), frozenset({37}), frozenset({46}), frozenset({42}), frozenset({5}), frozenset({15}), frozenset({12}), frozenset({4}), frozenset({38}), frozenset({2}), frozenset({34}), frozenset({20}), frozenset({7}), frozenset({3}), frozenset({29}), frozenset({6}), frozenset({19}), frozenset({21}), frozenset({35}), frozenset({0})}


{frozenset({0, 2, 46}),
 frozenset({10}),
 frozenset({25}),
 frozenset({30}),
 frozenset({4, 9}),
 frozenset({3, 18, 35}),
 frozenset({12, 31, 36, 48}),
 frozenset({39}),
 frozenset({8}),
 frozenset({27, 28}),
 frozenset({1, 19}),
 frozenset({13}),
 frozenset({7, 11, 37, 45}),
 frozenset({26}),
 frozenset({5, 22}),
 frozenset({17, 29, 47}),
 frozenset({38}),
 frozenset({33, 42}),
 frozenset({34}),
 frozenset({7, 15, 49}),
 frozenset({20}),
 frozenset({14, 44}),
 frozenset({6}),
 frozenset({23, 24, 40, 41, 43}),
 frozenset({21}),
 frozenset({16, 32, 45})}